In [1]:
import pandas as pd
data = {
    "name": ["John", "Ted", "Dev", "Brad", "Rex", "Smith", "Samuel", "David"],
    "salary": [10000, 20000, 50000, 45500, 19800, 95000, 5000, 50000],
    "management": [1, 0, 1, 0, 1, 0, 1, 0]}

# create dataframe from data dictionary
df = pd.DataFrame(data) # print the dataframe display(df.head())
df

,name,salary,management
0,John,10000,1
1,Ted,20000,0
2,Dev,50000,1
3,Brad,45500,0
4,Rex,19800,1
5,Smith,95000,0
6,Samuel,5000,1
7,David,50000,0


In [2]:
for index, row in df.iterrows():
    print(f"{index},{row['name']},{row['salary']},{row['management']}")

0,John,10000,1
1,Ted,20000,0
2,Dev,50000,1
3,Brad,45500,0
4,Rex,19800,1
5,Smith,95000,0
6,Samuel,5000,1
7,David,50000,0


In [3]:
# import projtool as pt

df_prompt_data = pd.read_csv("./data/prompt_log.csv")
df_prompt_data

,date,prompt_component,version,class,content,notes
0,2024-02-26,context,1,2,You are a fiction writer who has observed a di...,initial context for both classes used for send...
1,2024-02-26,prompt_prefix,1,0,"Write me a tweet similar to this one, under 14...",initial class 0 prompt used in web interface
2,2024-02-26,prompt_prefix,1,1,"Write me a tweet similar to this one, under 14...",initial class 1 prompt used in web interface
3,2024-02-27,context,2,2,You are a fiction writer who has observed a di...,same as version 1
4,2024-02-27,prompt_prefix,2,0,"Write me a tweet similar to this one, under 14...",avoid having to expand contractions on generat...
5,2024-02-27,prompt_prefix,2,1,"Write me a tweet similar to this one, under 14...",avoid having to expand contractions on generat...
6,2024-02-29,context,3,2,You are a fiction writer who has observed a di...,same as version 2
7,2024-02-29,prompt_prefix,3,0,"Write me a tweet similar to this one, under 14...",fix issue with generated tweet quoting
8,2024-02-29,prompt_prefix,3,1,"Write me a tweet similar to this one, under 14...",fix issue with generated tweet quoting
9,2024-03-23,context,4,0,You are social commentator who enjoys diverse ...,add context for class 0 generated tweets


In [4]:
max_context_date = df_prompt_data.loc[df_prompt_data['prompt_component'] == 'context', 'date'].max()
max_prefix_c0_date = df_prompt_data.loc[(df_prompt_data['prompt_component'] == 'prompt_prefix') & \
                                        (df_prompt_data['class'] == 0), 'date'].max()
max_prefix_c1_date = df_prompt_data.loc[(df_prompt_data['prompt_component'] == 'prompt_prefix') & \
                                        (df_prompt_data['class'] == 1), 'date'].max()
print(max_context_date, max_prefix_c0_date, max_prefix_c1_date)

max_context = df_prompt_data.loc[(df_prompt_data['prompt_component'] == 'context') & \
                                 (df_prompt_data['date'] == max_context_date), 'content'].values[0]
print(max_context)


2024-03-28 2024-03-28 2024-03-28
You are social commentator who enjoys diverse opinions and likes to tweet.


In [5]:
pd.set_option('display.max_colwidth', None)

In [6]:
from sklearn.utils.random import sample_without_replacement

# what happens if I sample multiple times w/o replacement?
print(sample_without_replacement(20, 5, random_state=42))
print(sample_without_replacement(20, 5, random_state=42))
print(sample_without_replacement(20, 5, random_state=42))
print(sample_without_replacement(20, 5, random_state=42))

[ 0 17 15  1  8]
[ 0 17 15  1  8]
[ 0 17 15  1  8]
[ 0 17 15  1  8]


In [7]:
n = str(4)
n.zfill(3)

'004'

In [8]:
# play with regex to fix double quoting issue
import regex as re

test_strings = ['"this starts and ends with double quote"', 'this has no quotes',
                '"starts with dq, but does" not end with dq', 'ends with dq, "but does not start with dq"',
                'has dq, but dq "does not start" or end']

# pattern_start_end_dq = '^"(.*)$"'
pattern_start_end_dq = r'^(")([^"]*)(")$'  # starts and ends with "
# patter_starts_dq = '^"(.*)
compiled_pattern = re.compile(pattern_start_end_dq)
match = compiled_pattern.match(test_strings[0])
print(match)
print(match.group(0))
print(match.group(1))
print(match.group(2))
print(match.group(3))

<regex.Match object; span=(0, 40), match='"this starts and ends with double quote"'>
"this starts and ends with double quote"
"
this starts and ends with double quote
"


In [9]:
# should not match 2nd string
match = compiled_pattern.match(test_strings[1])
if match is not None:
    print(match)
    print(match.group(0))
    print(match.group(1))
    print(match.group(2))
    print(match.group(3))


In [10]:
def fix_double_quotes(my_string):
    # if string starts and ends with ", just return it
    pattern_start_end_dq = re.compile(r'^(")([^"]*)(")$')
    # patter_starts_dq = '^"(.*)
    match_start_end_dq = pattern_start_end_dq.match(my_string)
    if match is not None:
        return_value = match_start_end_dq.group(0)

    pass

In [11]:
import projtools as pt

indices_dict = pt.get_batch_indices()
print(indices_dict)
print()
context0 = "You are social commentator who enjoys diverse opinions and likes to tweet."
context1 = "You are a world class reporter who has observed a disaster and likes to tweet."
prompt0 = "Write me a tweet similar to this one in length and content, under 141 characters, does not contain double quotes, but refers to a different activity, feeling and location: "
prompt1 = "Write me a tweet similar to this one in length and content, under 141 characters, does not contain double quotes, but refers to a different disaster and location: "
class0_tweets_test = pt.generate_tweet_batch(class_label=0, context_by_class=context0, start_prompt_by_class=prompt0, batch_start=400, batch_end=599)
class1_tweets_test = pt.generate_tweet_batch(class_label=1, context_by_class=context1, start_prompt_by_class=prompt1, batch_start=400, batch_end=599)

{'tweet_batches_class0': [(0, 199), (200, 399), (400, 599), (600, 799), (800, 999), (1000, 1199), (1200, 1399), (1400, 1599), (1600, 1799), (1800, 1999), (2000, 2199), (2200, 2399), (2400, 2599), (2600, 2799), (2800, 2999), (3000, 3199), (3200, 3399), (3400, 3599), (3600, 3799), (3800, 3999), (4000, 4199), (4200, 4297)], 'tweet_batches_class1': [(0, 199), (200, 399), (400, 599), (600, 799), (800, 999), (1000, 1199), (1200, 1399), (1400, 1599), (1600, 1799), (1800, 1999), (2000, 2199), (2200, 2399), (2400, 2599), (2600, 2799), (2800, 2999), (3100, 3189)]}

shape of train_csv_file: (7485, 4)
generating augmented tweets for class: 0
shape of df_by_class: (4297, 2)
processing row 10 with id 994
processing row 20 with id 1008
processing row 30 with id 1024
processing row 40 with id 1042
processing row 50 with id 1058
processing row 60 with id 1077
processing row 70 with id 1091
processing row 80 with id 1109
processing row 90 with id 1124
processing row 100 with id 1141
processing row 110 w